In [ ]:

!wget https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar -xvzf spark-3.2.1-bin-hadoop3.2.tgz
!ls /content/spark-3.2.1-bin-hadoop3.2

In [ ]:
!pip uninstall pyspark
!pip install pyspark==3.0.2

Found existing installation: pyspark 3.2.0
Uninstalling pyspark-3.2.0:
  Would remove:
    /usr/local/bin/beeline
    /usr/local/bin/beeline.cmd
    /usr/local/bin/docker-image-tool.sh
    /usr/local/bin/find-spark-home
    /usr/local/bin/find-spark-home.cmd
    /usr/local/bin/find_spark_home.py
    /usr/local/bin/load-spark-env.cmd
    /usr/local/bin/load-spark-env.sh
    /usr/local/bin/pyspark
    /usr/local/bin/pyspark.cmd
    /usr/local/bin/pyspark2.cmd
    /usr/local/bin/run-example
    /usr/local/bin/run-example.cmd
    /usr/local/bin/spark-class
    /usr/local/bin/spark-class.cmd
    /usr/local/bin/spark-class2.cmd
    /usr/local/bin/spark-shell
    /usr/local/bin/spark-shell.cmd
    /usr/local/bin/spark-shell2.cmd
    /usr/local/bin/spark-sql
    /usr/local/bin/spark-sql.cmd
    /usr/local/bin/spark-sql2.cmd
    /usr/local/bin/spark-submit
    /usr/local/bin/spark-submit.cmd
    /usr/local/bin/spark-submit2.cmd
    /usr/local/bin/sparkR
    /usr/local/bin/sparkR.cmd
    /usr/lo

In [ ]:
!pip install findspark
!pip install elephas
!pip install pyspark

In [ ]:
import os
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()

In [ ]:
# Spark Session, Pipeline, Functions, and Metrics
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
# from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import rand
from pyspark.mllib.evaluation import MulticlassMetrics

# Keras / Deep Learning
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras import optimizers, regularizers
from tensorflow.keras.optimizers import Adam 

import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,Conv1D,MaxPooling1D,LSTM
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
# Elephas for Deep Learning on Spark
from elephas.ml_model import ElephasEstimator
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
 
# for reproducibility


from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Dropout, SpatialDropout1D
from keras.layers import LSTM, Conv1D, MaxPooling1D
from sklearn.model_selection import train_test_split

In [ ]:
# Spark Session
conf = SparkConf().setAppName('Spark DL Tabular Pipeline').setMaster('local[6]')
sc = SparkContext(conf=conf)
sql_context = SQLContext(sc)

In [ ]:
# # Load Data to Spark Dataframe
# df = sql_context.read.csv('/bank.csv',
#                     header=True,
#                     inferSchema=True)
expr_name = 'expr_13'
look_back = 24*120 # 60 days, as each entry is for 1 hour
lstm_layers = 64
epochs=5
batch_size=64
num_of_features = 5
num_classes = 5

In [ ]:
csv = pd.read_csv("/content/df_data.csv")
csv.shape

(22420, 9)

In [ ]:
csv.head()

,Unnamed: 0,tweets,cleaned_tweets,date_clean,crypto_sentiment,subjectivity,polarity,sentiment,target
0,0,nice project \n\n\n\n\n\nortcoin ort okratech ...,nice project ortcoin ort okratech bitcoin aird...,2021-06-22,positive,1.00,0.600,positive,True
1,1,Long Bitcoin short the banks 🧐,Long Bitcoin short bank,2021-08-25,negative,0.35,-0.025,negative,True
2,2,Top Trending Cryptocurrency Post - DOGECOIN Se...,Top Trending Cryptocurrency Post DOGECOIN Sell...,2021-07-02,negative,0.40,0.250,positive,False
3,3,Can one expect another wave of BTC's decline s...,Can one expect another wave BTC decline soon v...,2021-07-24,negative,0.00,0.000,neutral,False
4,4,We will see…\n\nbitcoin btc bnb band bake $btc...,We see bitcoin btc bnb band bake btc dCc xZ dP,2021-05-29,positive,0.00,0.000,neutral,False


In [ ]:
from keras.layers import Dense
from keras.layers import LSTM, GRU
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd

In [ ]:
print('parsing CSV')

X, Y = [], []

for index, row in csv.iterrows():
    X.append(row[2])
    y_part = row[4]
    if y_part == "negative":
        yy = np.array([0])
    elif y_part == "positive":
        yy = np.array([1])
    # else:
    #     raise Exception('Invalid y_part value=' + y_part)
    Y.append(yy)

parsing CSV


In [ ]:
X

array([[    0,     0,     0, ...,  1735,     3, 13342],
       [    0,     0,     0, ...,     1,   116,   327],
       [    0,     0,     0, ..., 13343,     7,     1],
       ...,
       [    0,     0,     0, ...,   130,     2,     1],
       [    0,     0,     0, ...,  8883,   172, 11512],
       [    0,     0,     0, ...,    78, 41121,   119]], dtype=int32)

In [ ]:
print('build words map')

max_features = 50000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X)
X, Xt, Y, Yt = train_test_split(X, Y, test_size = 0.3, random_state = random.set_seed(1))

In [ ]:
import random
from tensorflow.random import set_seed

In [ ]:
X, Xt, Y, Yt = train_test_split(X, Y, test_size = 0.3, random_state = random.seed(1))
validation_size = 1500
X_validate = Xt[-validation_size:]
Y_validate = Yt[-validation_size:]
Xt = Xt[:-validation_size]
Yt = Yt[:-validation_size]

In [ ]:
maxlen = 0
def wrap_array(x, maxlen):
    for index in range(len(x)):
        xx = x[index]
        if len(xx) > maxlen:
            maxlen = len(xx)
        x[index] = np.array(xx)
    return np.array(x), maxlen

X, maxlen = wrap_array(X, maxlen)
Xt, maxlen = wrap_array(Xt, maxlen)
X_validate, maxlen = wrap_array(X_validate, maxlen)
Y, maxlen = wrap_array(Y, maxlen)
Yt, maxlen = wrap_array(Yt, maxlen)
Y_validate, maxlen = wrap_array(Y_validate, maxlen)

**Insert Model Here**

In [ ]:
print('build model')

batch_size = 256

model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(124, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])
print(model.summary())

# print('Train...')
# model.fit(X, Y, batch_size=batch_size, epochs=2, validation_data=(Xt, Yt), verbose=2)

build model
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 55, 128)           6400000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 55, 128)          0         
 ropout1D)                                                       
                                                                 
 conv1d (Conv1D)             (None, 55, 32)            12320     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 27, 32)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 124)               77872     
                                                                 
 dense (Dense)               (None, 1)      

In [ ]:
rdd = to_simple_rdd(sc, Xt, Yt)

In [ ]:
spark_model = SparkModel(model, frequency='epoch', num_workers=5, mode='synchronous')

In [ ]:
from pyspark import SparkContext, SparkConf
from keras.utils import np_utils
import time
import numpy as np

In [ ]:
!pip install py4j

In [ ]:
# Train Spark model
start_time = time.time()
spark_model.fit(rdd, epochs=2, batch_size=batch_size, verbose=10, validation_split=0.1)
end_time = time.time()
# Evaluate Spark model by evaluating the underlying model
score = spark_model.evaluate(Xt, Yt, verbose=2)
# print('Test accuracy:', score[1])

# np.savetxt('results.txt', [end_time-start_time, score[1]], fmt="%f")
# spark_model.save('mnist_model.h5')

>>> Fit model


Py4JError: ignored

**Data Preprocessing and Deep Learning Model**

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Build RDD from numpy features and labels
rdd = to_simple_rdd(sc, x_train, y_train)

# Initialize SparkModel from tensorflow.keras model and Spark context
spark_model = SparkModel(model, frequency='epoch', num_workers=5, mode='synchronous')

# Train Spark model
start_time = time.time()
spark_model.fit(rdd, epochs=epochs, batch_size=batch_size, verbose=10, validation_split=0.1)
end_time = time.time()
# Evaluate Spark model by evaluating the underlying model
score = spark_model.evaluate(x_test, y_test, verbose=2)
print('Test accuracy:', score[1])

np.savetxt('results.txt', [end_time-start_time, score[1]], fmt="%f")

**DL Model ends, Add Elephas**

In [ ]:
# Set and Serialize Optimizer
optimizer_conf = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)

# Initialize SparkML Estimator and Get Settings
estimator = ElephasEstimator()
estimator.setFeaturesCol("features")
estimator.setLabelCol("label_index")
estimator.set_keras_model_config(model.to_yaml())
estimator.set_categorical_labels(True)
estimator.set_nb_classes(nb_classes)
estimator.set_num_workers(1)
estimator.set_epochs(25) 
estimator.set_batch_size(64)
estimator.set_verbosity(1)
estimator.set_validation_split(0.10)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("synchronous")
estimator.set_loss("binary_crossentropy")
estimator.set_metrics(['acc'])

In [ ]:
# Create Deep Learning Pipeline
dl_pipeline = Pipeline(stages=[estimator])
def dl_pipeline_fit_score_results(dl_pipeline=dl_pipeline,
                                  train_data=train_data,
                                  test_data=test_data,
                                  label='label_index'):
    
    fit_dl_pipeline = dl_pipeline.fit(train_data)
    pred_train = fit_dl_pipeline.transform(train_data)
    pred_test = fit_dl_pipeline.transform(test_data)
    
    pnl_train = pred_train.select(label, "prediction")
    pnl_test = pred_test.select(label, "prediction")
    
    pred_and_label_train = pnl_train.rdd.map(lambda row: (row[label], row['prediction']))
    pred_and_label_test = pnl_test.rdd.map(lambda row: (row[label], row['prediction']))
    
    metrics_train = MulticlassMetrics(pred_and_label_train)
    metrics_test = MulticlassMetrics(pred_and_label_test)
    
    print("Training Data Accuracy: {}".format(round(metrics_train.precision(),4)))
    print("Training Data Confusion Matrix")
    display(pnl_train.crosstab('label_index', 'prediction').toPandas())
    
    print("\nTest Data Accuracy: {}".format(round(metrics_test.precision(),4)))
    print("Test Data Confusion Matrix")
    display(pnl_test.crosstab('label_index', 'prediction').toPandas())